In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(0)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/4from20data"))

# Any results you write to the current directory are saved as output.

['4from20.csv']


In [2]:
file = pd.read_csv("../input/4from20data/4from20.csv",encoding = 'unicode_escape',header=None)
file.columns = ['game','numbers'] 
cols = ['1_l', '2_l', '3_l', '4_l','1_r', '2_r', '3_r', '4_r']
file[cols[0]] = file['numbers'].apply(lambda x: int(x[0:2]))
file[cols[1]] = file['numbers'].apply(lambda x: int(x[3:5]))
file[cols[2]] = file['numbers'].apply(lambda x: int(x[6:8]))
file[cols[3]] = file['numbers'].apply(lambda x: int(x[9:11]))
file[cols[4]] = file['numbers'].apply(lambda x: int(x[12:14]))
file[cols[5]] = file['numbers'].apply(lambda x: int(x[15:17]))
file[cols[6]] = file['numbers'].apply(lambda x: int(x[18:20]))
file[cols[7]] = file['numbers'].apply(lambda x: int(x[21:23]))
file.head()

,game,numbers,1_l,2_l,3_l,4_l,1_r,2_r,3_r,4_r
0,1,20 02 10 04 02 16 09 17,20,2,10,4,2,16,9,17
1,2,12 06 20 17 03 16 09 13,12,6,20,17,3,16,9,13
2,3,05 19 18 17 14 11 20 12,5,19,18,17,14,11,20,12
3,4,19 17 12 05 03 08 07 06,19,17,12,5,3,8,7,6
4,5,03 13 08 09 05 16 07 08,3,13,8,9,5,16,7,8


In [3]:
df = file[cols]
print(len(df))
df.tail()

1191


,1_l,2_l,3_l,4_l,1_r,2_r,3_r,4_r
1186,18,10,19,16,18,1,17,3
1187,19,20,4,5,15,8,11,13
1188,1,8,11,5,19,13,2,3
1189,12,7,20,8,17,1,13,4
1190,6,4,18,1,16,7,5,17


In [4]:
dop_data = [
    [6,9,12,14,1,14,4,10],
]
df_dop = pd.DataFrame(dop_data, columns=df.columns)
df = df.append(df_dop, ignore_index=True)
df.tail()

,1_l,2_l,3_l,4_l,1_r,2_r,3_r,4_r
1187,19,20,4,5,15,8,11,13
1188,1,8,11,5,19,13,2,3
1189,12,7,20,8,17,1,13,4
1190,6,4,18,1,16,7,5,17
1191,6,9,12,14,1,14,4,10


In [5]:
# # SPN1 = np.zeros(shape=(100,4))
# # SPN2 = np.random.randint(1, 20, (1, 100, 4))

# bigmas1 = []
# for i in range(1163):
#     mas = []
#     while len(mas)<=3:
#         val = np.random.randint(1, 20)
#         if val not in mas:
#             mas.append(val)
#     bigmas1.append(mas)
    
# bigmas2 = []
# for i in range(1163):
#     mas = []
#     while len(mas)<=3:
#         val = np.random.randint(1, 20)
#         if val not in mas:
#             mas.append(val)
#     bigmas2.append(mas)

In [6]:
# df1 = pd.DataFrame(bigmas1,columns=['1_l', '2_l', '3_l', '4_l'])
# df2 = pd.DataFrame(bigmas2,columns=['1_r', '2_r', '3_r', '4_r'])
# df = pd.concat([df1,df2], axis = 1)
# df.head()

In [7]:
df1 = df[['1_l', '2_l', '3_l', '4_l']]
df2 = df[['1_r', '2_r', '3_r', '4_r']]
df1 = df1.drop(0)
df2 = df2.drop(len(df2)-1)
df1.index = df2.index
df_slice = pd.concat([df2,df1], axis = 1)
df_slice.columns = ['1_l', '2_l', '3_l', '4_l','1_r', '2_r', '3_r', '4_r']
df_slice.tail()

,1_l,2_l,3_l,4_l,1_r,2_r,3_r,4_r
1186,18,1,17,3,19,20,4,5
1187,15,8,11,13,1,8,11,5
1188,19,13,2,3,12,7,20,8
1189,17,1,13,4,6,4,18,1
1190,16,7,5,17,6,9,12,14


In [8]:
data = pd.concat([df,df_slice], axis = 0)
print (len(data))

2383


In [9]:
def intersection(boxes):
    assert(len(boxes) == 8)
    first = boxes[:4].values
    second = boxes[4:].values
    s = 0
    for el in second:
        if el in first:
            s = s + 1
    return s

In [10]:
data['iou'] = df_slice[['1_l', '2_l', '3_l', '4_l','1_r', '2_r', '3_r', '4_r']].apply(intersection, axis=1)
data.head()

,1_l,2_l,3_l,4_l,1_r,2_r,3_r,4_r,iou
0,20,2,10,4,2,16,9,17,1.0
1,12,6,20,17,3,16,9,13,0.0
2,5,19,18,17,14,11,20,12,1.0
3,19,17,12,5,3,8,7,6,2.0
4,3,13,8,9,5,16,7,8,0.0


In [11]:
from sklearn.model_selection import train_test_split

# X_train, X_valid, Y_train, Y_valid = train_test_split(data[['1_l', '2_l', '3_l', '4_l']],  data[['1_r', '2_r', '3_r', '4_r']], random_state=7, test_size=0.01)
X_train = data[['1_l', '2_l', '3_l', '4_l']]
Y_train = data[['1_r', '2_r', '3_r', '4_r']]
X_valid = data[['1_l', '2_l', '3_l', '4_l']][len(data)-100:len(data)]
Y_valid = data[['1_r', '2_r', '3_r', '4_r']][len(data)-100:len(data)]
X_test = pd.DataFrame([[1,14,4,10],[7,12,10,14]],columns=['1_l', '2_l', '3_l', '4_l'])

In [12]:
import xgboost
xgb = xgboost.XGBRegressor(n_estimators=300, learning_rate=0.2, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=35)
X = X_valid.copy()
Test = X_test.copy()
for el in ['1_r', '2_r', '3_r', '4_r']: 
    xgb.fit(X_train.values, Y_train[el].values)
    pred = xgb.predict(X_valid.values)
    pred2 = xgb.predict(X_test.values)
    X.index = pd.DataFrame(pred,columns=[el+'_PRED']).index
#     Test.index = pd.DataFrame(pred2,columns=[el+'_PRED']).index
    X = pd.concat([X, pd.DataFrame(pred,columns=[el+'_PRED'])], axis=1)
    Test = pd.concat([Test, pd.DataFrame(pred2,columns=[el+'_PRED'])], axis=1)
# iou_mean = X[['1_r_PRED', '2_r_PRED', '3_r_PRED', '4_r_PRED', '1_r', '2_r', '3_r', '4_r']].apply(intersection_over_union, axis=1).mean()
Y_valid.index = X.index
X = pd.concat([X, Y_valid], axis=1)
X = X.round(0)
X['iou'] = X[['1_r_PRED', '2_r_PRED', '3_r_PRED', '4_r_PRED', '1_r', '2_r', '3_r', '4_r']].apply(intersection, axis=1)
X['iou'].mean()

[19:12:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:12:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:12:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:12:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


3.95

In [13]:
Test = Test.round(0)
Test

,1_l,2_l,3_l,4_l,1_r_PRED,2_r_PRED,3_r_PRED,4_r_PRED
0,1,14,4,10,7.0,12.0,10.0,14.0
1,7,12,10,14,16.0,9.0,8.0,12.0
